In [87]:
from sklearn import svm
import mysql.connector
from mysql.connector import Error
from bs4 import BeautifulSoup
import re
import requests
import numpy as np

In [39]:
# Data source
url = 'https://www.scrapethissite.com/pages/simple/'
data_source = requests.get(url)

In [40]:
# MySQL configuration
DB_USER = 'test_user'
DB_PASS = 'StrongPass123!'
DB_HOST = '127.0.0.1'
DB_NAME = 'final'
TBL_NAME = 'countries'

In [41]:
config = {
        'user': DB_USER,
        'password': DB_PASS,
        'host': DB_HOST,
        # 'database': DB_NAME,
        'raise_on_warnings': True
          }

In [48]:
def read_source_data_content(data_source):
    soup = BeautifulSoup(data_source.text, 'html.parser')
    countries = soup.find_all('div', attrs= {'class' : 'col-md-4 country'})
    records = []
    for country in countries:
        country_name = country.find_all('h3', attrs= {'class' : 'country-name'})
        country_capital = country.find_all('span', attrs= {'class' : 'country-capital'})
        country_population = country.find_all('span', attrs= {'class' : 'country-population'})
        country_area = country.find_all('span', attrs= {'class' : 'country-area'})
        country_name_text = (re.sub(r'\s', ' ', country_name[0].text)).strip()
        country_capital_text = (re.sub(r'\s', ' ', country_capital[0].text)).strip()
        country_population_text = (re.sub(r'\s', ' ', country_population[0].text)).strip()
        country_area_text = (re.sub(r'\s', ' ', country_area[0].text)).strip()
        record = (country_name_text, country_capital_text, country_population_text, country_area_text)
        records.append(record)
    return records

In [49]:
def check_database_existance(connection):
    mycursor = connection.cursor(buffered = True)
    mycursor.execute("SHOW DATABASES")
    
    for db in mycursor:
        if db[0] == DB_NAME:
            return True
    return False

def check_table_exist(connection):
    mycursor = connection.cursor(buffered = True)
    mycursor.execute("SHOW TABLES")
    
    for tbl in mycursor:
        if tbl[0] == TBL_NAME:
            return True
    
    return False


In [50]:
def init_mysql():
    # Choosing db and connect to it
    conn = mysql.connector.connect(**config)
    cursor = conn.cursor(buffered= True)
    # Creating database if not exist
    if not check_database_existance(conn):
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
    
    conn.database = DB_NAME
    # Creating table if not exist
    if not check_table_exist(conn):
        cursor.execute(f"""
            CREATE TABLE IF NOT EXISTS {TBL_NAME} (
                country VARCHAR(255) PRIMARY KEY,
                capital VARCHAR(255) NOT NULL,
                population INT,
                area DECIMAL(10,1)
            )
        """)
    conn.commit()
    return conn


In [51]:
def convert_to_decimal_one(s):

    x = float(s)
    return round(x, 1)

def convert_to_integer(s):
    
    x = int(s)
    return x

In [56]:
try:
    records = read_source_data_content(data_source)
    conn = init_mysql()
    cursor = conn.cursor(buffered=True)
    insert_query = '''INSERT INTO countries (country, capital, population, area)
                    VALUES (%s, %s, %s, %s)
                    '''

    for name, capital, population, area in records:
        try:
            population = convert_to_integer(population)
        except ValueError:
            print(f'Wrong value for {name} population: {population}')
            
        try:
            area = convert_to_decimal_one(area)
        except ValueError:
            print(f'Wrong value for {name} area: {area}')

        try:
            cursor.execute(insert_query, (name, capital, population, area))
        except Error as e:
            if e.errno == 1062 and "Duplicate entry" in str(e):
                # clear_output()
                # print('Country already exist')
                continue
            else:
                print(f'Error in {name}: {e}')

except Error as conn_e:
    print('Connection to MySQL error:', conn_e)
# finally:
#     if conn.is_connected():
#         conn.commit()
#         cursor.close()
#         conn.close()
#         print('Connection to database closed')

In [62]:
cursor = conn.cursor(buffered=True)
cursor.execute('SELECT population,area FROM countries')
result = cursor.fetchall()
print(result)

[(29121286, Decimal('647500.0')), (26711, Decimal('1580.0')), (2986952, Decimal('28748.0')), (34586184, Decimal('2381740.0')), (57881, Decimal('199.0')), (84000, Decimal('468.0')), (13068161, Decimal('1246700.0')), (13254, Decimal('102.0')), (0, Decimal('14000000.0')), (86754, Decimal('443.0')), (41343201, Decimal('2766890.0')), (2968000, Decimal('29800.0')), (71566, Decimal('193.0')), (21515754, Decimal('7686850.0')), (8205000, Decimal('83858.0')), (8303512, Decimal('86600.0')), (301790, Decimal('13940.0')), (738004, Decimal('665.0')), (156118464, Decimal('144000.0')), (285653, Decimal('431.0')), (9685000, Decimal('207600.0')), (10403000, Decimal('30510.0')), (314522, Decimal('22966.0')), (9056010, Decimal('112620.0')), (65365, Decimal('53.0')), (699847, Decimal('47000.0')), (9947418, Decimal('1098580.0')), (18012, Decimal('328.0')), (4590000, Decimal('51129.0')), (2029307, Decimal('600370.0')), (0, Decimal('49.0')), (201103330, Decimal('8511965.0')), (4000, Decimal('60.0')), (21730, 

In [79]:
x = []
y = []
for row in result:
    x.append(row[0])
    y.append(row[1])
x= np.array(x).reshape(-1, 1)
y = np.array(y)

In [88]:
reg = svm.SVR()
reg.fit(x, y)

,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,tol,0.001
,C,1.0
,epsilon,0.1
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


In [94]:
reg.predict([[800000]])

array([64893.94782423])

In [96]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

reg = make_pipeline(StandardScaler(), SGDRegressor(max_iter=1000, tol=1e-3))
reg.fit(x,y)

,steps,"[('standardscaler', ...), ('sgdregressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,loss,'squared_error'
,penalty,'l2'
,alpha,0.0001
,l1_ratio,0.15


In [97]:
reg.predict([[8000000], [80000000]])

array([472418.33609665, 951882.28570203])